# Validation Etapes 1 a 3 (vue metier)

Ce notebook explique le projet de facon pedagogique pour un profil metier.

Objectif: verifier de bout en bout que le socle technique fonctionne pour un assistant de recommandation culturelle:
1. Environnement (Etape 1)
2. Donnees OpenAgenda preparees (Etape 2)
3. Indexation FAISS et recherche locale (Etape 3)


## Comment lire ce notebook

- Les cellules `Markdown` expliquent le pourquoi metier.
- Les cellules `Code` prouvent que ca fonctionne.
- Les graphiques donnent une lecture rapide de la qualite du pipeline.

Conseil: faire `Restart Kernel` puis `Run All`.


In [1]:
from __future__ import annotations

import hashlib
import importlib.metadata as md
import json
import os
import subprocess
import sys
import tempfile
from datetime import date, timedelta
from pathlib import Path

import pandas as pd
import yaml
from IPython.display import display

try:
    import matplotlib.pyplot as plt
    MATPLOTLIB_OK = True
except Exception:
    MATPLOTLIB_OK = False


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'config.yaml').exists() and (candidate / 'src').exists():
            return candidate
    raise RuntimeError('Impossible de trouver la racine du projet.')


def draw_bar(labels, values, title, ylabel='Valeur', color='#1f77b4'):
    if not MATPLOTLIB_OK:
        print(f'[visualisation ignoree] {title} (matplotlib non disponible)')
        return
    fig, ax = plt.subplots(figsize=(9, 3.5))
    ax.bar(labels, values, color=color)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.grid(axis='y', alpha=0.25)
    plt.xticks(rotation=20, ha='right')
    plt.tight_layout()
    plt.show()


PROJECT_ROOT = find_project_root(Path.cwd())
os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print('PROJECT_ROOT =', PROJECT_ROOT)
print('Matplotlib disponible =', MATPLOTLIB_OK)


PROJECT_ROOT = /Users/steph/Code/Python/Jupyter/OCR_projet09
Matplotlib disponible = False


## 1) Etape 1 - Sante de l'environnement

Lecture metier: avant de parler IA, on verifie que les briques de base sont presentes.
Si cette etape echoue, tout le reste est fragile.


In [2]:
components = [
    ('python', None),
    ('langchain', 'langchain'),
    ('faiss-cpu', 'faiss-cpu'),
    ('mistralai', 'mistralai'),
    ('pandas', 'pandas'),
    ('requests', 'requests'),
]

rows = []
for label, dist_name in components:
    if label == 'python':
        rows.append({'component': label, 'version': f'{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}', 'installed': True})
        continue
    try:
        version = md.version(dist_name)
        rows.append({'component': label, 'version': version, 'installed': True})
    except md.PackageNotFoundError:
        rows.append({'component': label, 'version': 'not installed', 'installed': False})

env_df = pd.DataFrame(rows)
display(env_df)

checks = [
    ('import faiss', 'import faiss'),
    ('from langchain.vectorstores import FAISS', 'from langchain.vectorstores import FAISS'),
    ('from langchain.embeddings import HuggingFaceEmbeddings', 'from langchain.embeddings import HuggingFaceEmbeddings'),
    ('from mistral import MistralClient', 'from mistral import MistralClient'),
]

import_rows = []
for label, stmt in checks:
    try:
        exec(stmt, {})
        import_rows.append({'check': label, 'status': 'OK', 'details': ''})
    except Exception as exc:
        import_rows.append({'check': label, 'status': 'KO', 'details': str(exc)})

imports_df = pd.DataFrame(import_rows)
display(imports_df)

ok_count = int((imports_df['status'] == 'OK').sum())
ko_count = int((imports_df['status'] == 'KO').sum())
draw_bar(['Imports OK', 'Imports KO'], [ok_count, ko_count], 'Etape 1 - Etat des imports critiques', ylabel='Nombre')

assert ko_count == 0, 'Des imports critiques sont en echec.'
print('Etape 1 validee: environnement operationnel.')


,component,version,installed
0,python,3.11.3,True
1,langchain,0.0.353,True
2,faiss-cpu,1.13.2,True
3,mistralai,0.4.2,True
4,pandas,2.2.2,True
5,requests,2.32.3,True


,check,status,details
0,import faiss,OK,
1,from langchain.vectorstores import FAISS,OK,
2,from langchain.embeddings import HuggingFaceEm...,OK,
3,from mistral import MistralClient,OK,


[visualisation ignoree] Etape 1 - Etat des imports critiques (matplotlib non disponible)
Etape 1 validee: environnement operationnel.


## 2) Etape 2 - Perimetre metier et configuration

Lecture metier: on confirme ce qui est couvert (zone geographique, periode, langue).
Cela garantit que les recommandations restent dans le cadre attendu par Puls-Events.


In [3]:
config = yaml.safe_load(Path('config.yaml').read_text(encoding='utf-8'))
oa = config['openagenda']

today = date.today()
start_date = oa['time_window'].get('start_date') or str(today - timedelta(days=365))
end_date = oa['time_window'].get('end_date') or str(today + timedelta(days=90))

summary_df = pd.DataFrame([
    {'champ': 'Departement cible', 'valeur': oa['location'].get('department', '')},
    {'champ': 'Ville pivot', 'valeur': oa['location'].get('city', '')},
    {'champ': 'Latitude', 'valeur': oa['location'].get('latitude', '')},
    {'champ': 'Longitude', 'valeur': oa['location'].get('longitude', '')},
    {'champ': 'Rayon km', 'valeur': oa['location'].get('radius_km', '')},
    {'champ': 'Langue', 'valeur': oa['request'].get('language', '')},
    {'champ': 'Periode debut', 'valeur': start_date},
    {'champ': 'Periode fin', 'valeur': end_date},
])
display(summary_df)

start_dt = pd.to_datetime(start_date)
today_dt = pd.to_datetime(str(today))
end_dt = pd.to_datetime(end_date)

if MATPLOTLIB_OK:
    fig, ax = plt.subplots(figsize=(10, 2.3))
    ax.hlines(y=1, xmin=start_dt, xmax=end_dt, linewidth=8, color='#4c78a8', alpha=0.35)
    ax.scatter([start_dt, today_dt, end_dt], [1, 1, 1], s=90, c=['#2ca02c', '#ff7f0e', '#d62728'])
    ax.text(start_dt, 1.03, 'debut', ha='left')
    ax.text(today_dt, 1.03, 'aujourdhui', ha='center')
    ax.text(end_dt, 1.03, 'fin', ha='right')
    ax.set_title('Fenetre temporelle de collecte des evenements')
    ax.set_yticks([])
    ax.grid(axis='x', alpha=0.25)
    plt.tight_layout()
    plt.show()

assert oa['location'].get('department') == '34'
assert oa['request'].get('language') == 'fr'
print('Etape 2 - perimetre metier valide.')


,champ,valeur
0,Departement cible,34
1,Ville pivot,Montpellier
2,Latitude,43.6119
3,Longitude,3.8772
4,Rayon km,55
5,Langue,fr
6,Periode debut,2025-02-10
7,Periode fin,2026-05-11


Etape 2 - perimetre metier valide.


## 3) Etape 2 - Recuperation OpenAgenda (mockee, sans reseau)

Lecture metier: on simule des pages API pour prouver que le mecanisme de pagination est fiable.

Pourquoi un mock ? Pour des tests rapides, relancables et independants d'Internet.


In [4]:
from src.openagenda.client import OpenAgendaConfig, fetch_events

class FakeResponse:
    def __init__(self, payload, status_code=200):
        self._payload = payload
        self.status_code = status_code
        self.text = str(payload)

    def json(self):
        return self._payload

class FakeSession:
    def __init__(self, responses):
        self.responses = list(responses)
        self.call_count = 0

    def get(self, url, params=None, timeout=None):
        del url, params, timeout
        self.call_count += 1
        if not self.responses:
            return FakeResponse({'events': []})
        return FakeResponse(self.responses.pop(0))

    def close(self):
        pass

pages = [
    {'events': [{'uid': 'evt-1', 'title': {'fr': 'Evenement 1'}}]},
    {'events': [{'uid': 'evt-2', 'title': {'fr': 'Evenement 2'}}]},
    {'events': []},
]
fake_session = FakeSession(list(pages))

cfg = OpenAgendaConfig(
    base_url='https://api.openagenda.com/v2/events',
    api_key='demo-key',
    department='34',
    city='Montpellier',
    page_size=1,
    max_pages=5,
    max_events=10,
    start_date='2025-01-01',
    end_date='2026-12-31',
)

raw_events = fetch_events(cfg, session=fake_session)
print('Evenements recuperes:', len(raw_events))
print('UIDs:', [evt.get('uid') for evt in raw_events])

page_counts = [len(page.get('events', [])) for page in pages]
draw_bar([f'page_{i+1}' for i in range(len(page_counts))], page_counts, 'Simulation pagination API (events par page)', ylabel='Events')

assert len(raw_events) == 2
assert fake_session.call_count == 3
print('Pagination mockee validee.')


Evenements recuperes: 2
UIDs: ['evt-1', 'evt-2']
[visualisation ignoree] Simulation pagination API (events par page) (matplotlib non disponible)
Pagination mockee validee.


## 4) Etape 2 - Nettoyage et structuration des donnees

Lecture metier: on transforme des donnees brutes en donnees exploitables par le moteur de recherche.

Ce qui est mesure ici:
- donnees hors periode
- donnees incompletes
- doublons
- volume final de donnees propres


In [5]:
from src.preprocess.cleaning import clean_events
from src.preprocess.schema import EVENT_RECORD_FIELDS

demo_raw_events = [
    {
        'uid': 'evt-valid',
        'title': {'fr': 'Concert test'},
        'description': {'fr': 'Soiree musicale'},
        'firstTiming': {'begin': '2025-06-01T19:00:00Z', 'end': '2025-06-01T21:00:00Z'},
        'location': {'name': {'fr': 'Salle A'}, 'address': '10 rue de la Paix', 'city': 'Montpellier', 'latitude': 43.6119, 'longitude': 3.8772},
        'canonicalUrl': 'https://example.org/events/evt-valid',
        'tags': ['musique', 'culture'],
    },
    {
        'uid': 'evt-valid',
        'title': {'fr': 'Concert test'},
        'description': {'fr': 'Soiree musicale'},
        'firstTiming': {'begin': '2025-06-01T19:00:00Z', 'end': '2025-06-01T21:00:00Z'},
        'location': {'name': {'fr': 'Salle A'}, 'city': 'Montpellier'},
    },
    {
        'uid': 'evt-old',
        'title': {'fr': 'Ancien evenement'},
        'firstTiming': {'begin': '2023-01-01T10:00:00Z'},
        'location': {'city': 'Montpellier'},
    },
    {
        'uid': 'evt-missing-title',
        'title': {'fr': ''},
        'firstTiming': {'begin': '2025-06-02T10:00:00Z'},
        'location': {'city': 'Montpellier'},
    },
]

processed, stats = clean_events(
    raw_events=demo_raw_events,
    start_date='2025-01-01',
    end_date='2026-01-31',
    language='fr',
    source='openagenda',
)

stats_df = pd.DataFrame([{'metric': k, 'value': v} for k, v in stats.items()])
display(stats_df)

clean_df = pd.DataFrame(processed)
display(clean_df.head())

pipeline_labels = ['raw_events', 'after_period_filter', 'processed_events']
pipeline_values = [stats['raw_events'], stats['after_period_filter'], stats['processed_events']]
draw_bar(pipeline_labels, pipeline_values, 'Pipeline qualite donnees (avant/apres)', ylabel='Nombre de records', color='#2ca02c')

if not clean_df.empty and 'city' in clean_df.columns:
    city_counts = clean_df['city'].value_counts()
    draw_bar(list(city_counts.index), list(city_counts.values), 'Distribution geographique des records propres', ylabel='Records', color='#ff7f0e')

assert stats['processed_events'] == len(processed)
assert all(col in clean_df.columns for col in EVENT_RECORD_FIELDS)
print('Cleaning + schema valides.')


,metric,value
0,raw_events,4
1,missing_required,1
2,outside_period,1
3,after_period_filter,2
4,duplicates_removed,1
5,invalid_records,0
6,processed_events,1


,event_id,title,description,start_datetime,end_datetime,city,location_name,address,latitude,longitude,url,tags,source,document_text,retrieval_metadata
0,evt-valid,Concert test,Soiree musicale,2025-06-01T19:00:00Z,2025-06-01T21:00:00Z,Montpellier,Salle A,10 rue de la Paix,43.6119,3.8772,https://example.org/events/evt-valid,"[musique, culture]",openagenda,Titre: Concert test\nDescription: Soiree music...,"{'event_id': 'evt-valid', 'city': 'Montpellier..."


[visualisation ignoree] Pipeline qualite donnees (avant/apres) (matplotlib non disponible)
[visualisation ignoree] Distribution geographique des records propres (matplotlib non disponible)
Cleaning + schema valides.


## 5) Etape 2 - Simulation des sorties fichiers

Lecture metier: on verifie qu'un utilisateur peut relancer le pipeline et obtenir les artefacts attendus.


In [6]:
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp = Path(tmp_dir)
    raw_path = tmp / 'events_raw.jsonl'
    processed_path = tmp / 'events_processed.parquet'

    with raw_path.open('w', encoding='utf-8') as f:
        for item in demo_raw_events:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

    pd.DataFrame(processed).to_parquet(processed_path, index=False)

    file_stats = pd.DataFrame([
        {'fichier': raw_path.name, 'taille_octets': raw_path.stat().st_size},
        {'fichier': processed_path.name, 'taille_octets': processed_path.stat().st_size},
    ])
    display(file_stats)
    draw_bar(list(file_stats['fichier']), list(file_stats['taille_octets']), 'Taille des sorties generees', ylabel='Octets', color='#9467bd')

print('Sorties fichiers validees.')


,fichier,taille_octets
0,events_raw.jsonl,924
1,events_processed.parquet,14047


[visualisation ignoree] Taille des sorties generees (matplotlib non disponible)
Sorties fichiers validees.


## 6) Etape 3 - Indexation FAISS et recherche

Lecture metier: cette etape transforme les textes en vecteurs numeriques pour retrouver vite les evenements pertinents.

Ce que nous validons ici:
- chunking (decoupage en segments de texte)
- construction de l'index FAISS
- persistence/reload de l'index
- recherche locale top-k


In [7]:
from src.indexing.chunking import ChunkingConfig, split_text_into_chunks
from src.indexing.build_index import build_faiss_index, load_faiss_index
from src.indexing.search import search_similar_chunks

try:
    import faiss  # noqa: F401
except Exception as exc:
    raise RuntimeError('FAISS indisponible. Verifie l installation des dependances.') from exc

class FakeEmbeddings:
    """Embeddings deterministes pour test offline."""

    def __init__(self, dimension: int = 64) -> None:
        self.dimension = dimension

    def _embed(self, text: str):
        vector = [0.0] * self.dimension
        for idx, token in enumerate(text.lower().split()):
            digest = hashlib.sha256(token.encode('utf-8')).digest()
            bucket = digest[0] % self.dimension
            vector[(bucket + idx) % self.dimension] += (digest[1] / 255.0) + 0.01
        norm = sum(v * v for v in vector) ** 0.5 or 1.0
        return [v / norm for v in vector]

    def embed_documents(self, texts):
        return [self._embed(text) for text in texts]

    def embed_query(self, text):
        return self._embed(text)

    def __call__(self, text):
        return self.embed_query(text)

# 6.1 Chunking pedagogique
chunk_cfg = ChunkingConfig(chunk_size=140, chunk_overlap=20, min_chunk_size=30)
chunk_text = ('Concert jazz a Montpellier. ' * 60).strip()
chunks = split_text_into_chunks(chunk_text, chunk_cfg)
chunk_lengths = [len(c) for c in chunks]
print('Nombre de chunks:', len(chunks))

if MATPLOTLIB_OK:
    fig, ax = plt.subplots(figsize=(8, 3.2))
    ax.hist(chunk_lengths, bins=min(10, len(chunk_lengths)), color='#17becf', edgecolor='black', alpha=0.8)
    ax.set_title('Distribution de la taille des chunks')
    ax.set_xlabel('Nombre de caracteres')
    ax.set_ylabel('Frequence')
    ax.grid(alpha=0.2)
    plt.tight_layout()
    plt.show()

assert len(chunks) >= 2

# 6.2 Build + save + reload + query
mini_df = pd.DataFrame([
    {
        'event_id': 'evt-jazz',
        'title': 'Concert jazz Montpellier',
        'document_text': 'Concert jazz en plein air a Montpellier avec trio local.',
        'start_datetime': '2025-07-12T20:00:00Z',
        'end_datetime': '2025-07-12T22:00:00Z',
        'city': 'Montpellier',
        'location_name': 'Place de la Comedie',
        'url': 'https://example.org/evt-jazz',
        'source': 'openagenda',
        'retrieval_metadata': {'department': '34', 'tags': ['jazz', 'musique']},
    },
    {
        'event_id': 'evt-theatre',
        'title': 'Theatre contemporain',
        'document_text': 'Representation de theatre contemporain a Sete.',
        'start_datetime': '2025-07-20T19:00:00Z',
        'end_datetime': '2025-07-20T21:00:00Z',
        'city': 'Sete',
        'location_name': 'Scene nationale',
        'url': 'https://example.org/evt-theatre',
        'source': 'openagenda',
        'retrieval_metadata': {'department': '34', 'tags': ['theatre']},
    },
    {
        'event_id': 'evt-photo',
        'title': 'Expo photo',
        'document_text': 'Exposition photographique sur le patrimoine mediterraneen.',
        'start_datetime': '2025-08-04T10:00:00Z',
        'end_datetime': '2025-08-04T18:00:00Z',
        'city': 'Beziers',
        'location_name': 'Galerie municipale',
        'url': 'https://example.org/evt-photo',
        'source': 'openagenda',
        'retrieval_metadata': {'department': '34', 'tags': ['photo']},
    },
])

with tempfile.TemporaryDirectory() as tmp_dir:
    tmp = Path(tmp_dir)
    input_path = tmp / 'events.parquet'
    index_dir = tmp / 'faiss_index'
    mini_df.to_parquet(input_path, index=False)

    indexing_cfg = {
        'paths': {'input_dataset': str(input_path), 'output_dir': str(index_dir)},
        'chunking': {'chunk_size': 180, 'chunk_overlap': 30, 'min_chunk_size': 40, 'separators': ['\n\n', '\n', '. ', ' ', '']},
        'embeddings': {'provider': 'huggingface', 'huggingface_model': 'intfloat/multilingual-e5-small', 'mistral_model': 'mistral-embed'},
        'faiss': {'normalize_L2': True},
        'search': {'top_k': 3},
    }

    fake_embeddings = FakeEmbeddings()
    result = build_faiss_index(
        input_path=input_path,
        output_dir=index_dir,
        config=indexing_cfg,
        embedding_model=fake_embeddings,
    )

    assert result.events_input == 3
    assert result.chunks_count >= 3
    assert (index_dir / 'index.faiss').exists()
    assert (index_dir / 'index.pkl').exists()
    assert (index_dir / 'index_metadata.json').exists()

    store = load_faiss_index(index_dir=index_dir, config=indexing_cfg, embedding_model=fake_embeddings)
    docs = store.similarity_search('concert jazz montpellier', k=2)
    assert len(docs) == 2

    results = search_similar_chunks(
        query='Je cherche un concert jazz a Montpellier',
        index_dir=index_dir,
        k=3,
        config=indexing_cfg,
        embedding_model=fake_embeddings,
    )
    results_df = pd.DataFrame([
        {
            'rank': item['rank'],
            'score': item['score'],
            'event_id': item['metadata'].get('event_id', ''),
            'city': item['metadata'].get('city', ''),
            'start_datetime': item['metadata'].get('start_datetime', ''),
        }
        for item in results
    ])
    display(results_df)

    top_event = results[0]['metadata'].get('event_id') if results else ''
    print('Top event_id:', top_event)
    assert top_event == 'evt-jazz'

    if MATPLOTLIB_OK and not results_df.empty:
        scores = [float(x) if x is not None else 0.0 for x in results_df['score']]
        draw_bar([f"rank_{r}" for r in results_df['rank']], scores, 'Score de similarite des top resultats', ylabel='Score', color='#bcbd22')

print('Etape 3 validee: index FAISS operationnel.')


Nombre de chunks: 13


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


,rank,score,event_id,city,start_datetime
0,1,0.991341,evt-jazz,Montpellier,2025-07-12T20:00:00Z
1,2,1.735267,evt-theatre,Sete,2025-07-20T19:00:00Z
2,3,2.000000,evt-photo,Beziers,2025-08-04T10:00:00Z


Top event_id: evt-jazz
Etape 3 validee: index FAISS operationnel.


## 7) Validation finale (scripts projet)

Lecture metier: cette section execute les verifications officielles du depot.

- `scripts/check_env.py` : controle environnement
- `pytest -q` : tests unitaires relancables


In [8]:
commands = [
    ['python3', 'scripts/check_env.py'],
    ['pytest', '-q'],
]

for cmd in commands:
    print('\n$ ' + ' '.join(cmd))
    completed = subprocess.run(cmd, text=True, capture_output=True)
    print(completed.stdout)
    if completed.returncode != 0:
        print(completed.stderr)
        raise RuntimeError(f"Commande en echec: {' '.join(cmd)}")

print('Validation globale Etapes 1 a 3: OK')



$ python3 scripts/check_env.py
=== Environment versions ===
python: 3.11.3
langchain: 0.0.353
faiss-cpu: 1.13.2
mistralai: 0.4.2
pandas: 2.2.2
requests: 2.32.3

=== Import checks ===
[OK] import faiss
[OK] from langchain.vectorstores import FAISS
[OK] from langchain.embeddings import HuggingFaceEmbeddings
[OK] from mistral import MistralClient

[SUCCESS] Environment smoke test passed.


$ pytest -q
...........                                                              [100%]
11 passed in 0.72s

Validation globale Etapes 1 a 3: OK
